# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686543


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:40,  1.45s/it]

Rendering models:  10%|█         | 3/30 [00:03<00:29,  1.08s/it]

Rendering models:  17%|█▋        | 5/30 [00:03<00:20,  1.25it/s]

Rendering models:  20%|██        | 6/30 [00:03<00:15,  1.59it/s]

...

Rendering models:  83%|████████▎ | 25/30 [00:11<00:01,  3.67it/s]

Rendering models:  87%|████████▋ | 26/30 [00:12<00:01,  3.60it/s]

Rendering models:  90%|█████████ | 27/30 [00:12<00:00,  3.08it/s]

Rendering models:  97%|█████████▋| 29/30 [00:13<00:00,  3.62it/s]

not-logged-in-bc5fdb9ea8d611e60b8b    0.000242
ethanscamander                        0.000817
austingasbarra                        0.000306
kjkavene                              0.000217
jmfranci                              0.000953
tosnyder                              0.000481
not-logged-in-68ca6f2b89baff5eebf6    0.000180
not-logged-in-a3d7379de3dae98540ca    0.000772
not-logged-in-81bd7d1854a2e6d7ab22    0.021451
Giantjack14                           0.012327
LordNeveris                           0.000334
not-logged-in-44beefa7ea1fce864d18    0.000203
clairedeu                             0.000329
not-logged-in-300dca7f45d1a5d1dd58    0.000428
Ellaaa666                             0.001559
pangeli5                              0.000561
not-logged-in-fcfcbc88feb1aa0ed0a7    0.000776
Lavadude                              0.005839
scoripyo                              0.000263
Gatto_Volante                         0.000196
mhaloftis                             0.000738
Mkcls        

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())